<a href="https://colab.research.google.com/github/Xenofon-Chiotopoulos/Qhack-2022/blob/main/QHack.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
!pip install pennylane
import pennylane as qml
from pennylane import numpy as np
from pennylane import hf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 957 kB 5.1 MB/s 
     |████████████████████████████████| 1.6 MB 23.7 MB/s 
     |████████████████████████████████| 9.3 MB 10.0 MB/s 
     |████████████████████████████████| 108 kB 49.9 MB/s 


# QML

In [ ]:
n_qubits = 2
dev = qml.device("default.qubit", wires=n_qubits)

@qml.qnode(dev)
def generating_fourier_state(n_qubits, m):
    """Function which, given the number of qubits and an integer m, returns the circuit and the angles that generate the state
    QFT|m> following the above template.

    Args:
        - n_qubits (int): number of qubits in the circuit.
        - m (int): basis state that we generate. For example, for 'm = 3' and 'n_qubits = 4'
        we would generate the state QFT|0011> (3 in binary is 11).

    Returns:
       - (qml.QNode): circuit used to generate the state.
       - (list[float]): angles that generate the state QFT|m>.
    """

    dev = qml.device("default.qubit", wires=n_qubits)

    @qml.qnode(dev)
    def circuit(angles):
        """This is the quantum circuit that we will use."""

        # QHACK #

        # Add the template of the statement with the angles passed as an argument.
        for i in range(n_qubits):
            qml.Hadamard(wires = i)
            qml.RZ(angles[i], wires=i)
        # QHACK #

        # We apply QFT^-1 to return to the computational basis.
        # This will help us to see how well we have done.
        qml.adjoint(qml.QFT)(wires=range(n_qubits))

        # We return the probabilities of seeing each basis state.
        return qml.probs(wires=range(n_qubits))

    def error(angles):
        """This function will determine, given a set of angles, how well it approximates
        the desired state. Here it will be necessary to call the circuit to work with these results.
        """

        probs = circuit(angles)
        # QHACK #
        err = np.zeros(n_qubits**n_qubits)
        err[m-1] = 0.5
        err[2*n_qubits-1] = 0.5
        # The return error should be smaller when the state m is more likely to be obtained.
        return abs(err-probs)

    opt = qml.AdamOptimizer(stepsize=0.8)
    epochs = 5000

    angles = np.zeros(n_qubits, requires_grad=True)

    for epoch in range(epochs):
        angles = opt.step(error, angles)
        angles = np.clip(opt.step(error, angles), -2 * np.pi, 2 * np.pi)

    return circuit, angles

In [ ]:
generating_fourier_state(2,3)

TypeError: ignored

In [ ]:
dev = qml.device("default.qubit", wires=n_qubits)
@qml.qnode(dev)
def circuit(angles):
    """This is the quantum circuit that we will use."""

    # QHACK #

    # Add the template of the statement with the angles passed as an argument.
    for i in range(n_qubits):
        qml.Hadamard(wires = i)
        qml.RZ(angles[i], wires=i)
    # QHACK #

    # We apply QFT^-1 to return to the computational basis.
    # This will help us to see how well we have done.
    qml.adjoint(qml.QFT)(wires=range(n_qubits))

    # We return the probabilities of seeing each basis state.
    return qml.probs(wires=range(n_qubits))
def error(angles):
    """This function will determine, given a set of angles, how well it approximates
    the desired state. Here it will be necessary to call the circuit to work with these results.
    """

    probs = circuit(angles)
    # QHACK #

    # The return error should be smaller when the state m is more likely to be obtained.
    def binary_list(m, n):
        """Converts number m to binary encoded on a list of length n

        Args:
            - m (int): Number to convert to binary
            - n (int): Number of wires in the circuit

        Returns:
            - (list(int)): Binary stored as a list of length n
        """
    """
        arr = [int(i) for i in bin(m)[2:]]
        difference = n - len(arr)
        for i in range(difference):
            arr.insert(i, 0)

        return arr
    dummy = binary_list(m, n_qubits)
    state_vec = []
    state_vec.append(np.array(dummy))
    sum = np.sum(state_vec)
    """
    err = np.zeros(n_qubits**n_qubits)
    err[m-1] = 0.5
    err[2*n_qubits-1] = 0.5
    return print(abs(err-probs))

In [ ]:
m = 3
angles = np.zeros(n_qubits, requires_grad=True)
error(angles)

[1.00000000e+00 1.10047289e-33 5.00000000e-01 5.00000000e-01]


#Quantum chemistry 100

In [ ]:
def binary_list(m, n):
    """Converts number m to binary encoded on a list of length n

    Args:
        - m (int): Number to convert to binary
        - n (int): Number of wires in the circuit

    Returns:
        - (list(int)): Binary stored as a list of length n
    """

    arr = [int(i) for i in bin(m)[2:]]
    difference = n - len(arr)
    for i in range(difference):
        arr.insert(i, 0)

    return arr

In [ ]:
binary_list(7, 4)

[0, 1, 1, 1]

In [ ]:
def basis_states(n):
    """Given a number n, returns a list of all binary_list(m,n) for m < 2**n, thus providing all basis states
         for a circuit of n wires

    Args:
        - n(int): integer representing the number of wires in the circuit

    Returns:
        - (list(list(int))): list of basis states represented as lists of 0s and 1s.
    """
    arrays = []
    for i in range(n):
        array = np.zeros(n)
        array[i] = 1
        arrays.append(array)

    return arrays

In [ ]:
basis_states(4)

[tensor([1., 0., 0., 0.], requires_grad=True),
 tensor([0., 1., 0., 0.], requires_grad=True),
 tensor([0., 0., 1., 0.], requires_grad=True),
 tensor([0., 0., 0., 1.], requires_grad=True)]

In [ ]:
def is_particle_preserving(circuit, n):
    """Given a circuit and its number of wires n, returns 1 if it preserves the number of particles, and 0 if it does not

    Args:
        - circuit (qml.QNode): A QNode that has a state such as [0,0,1,0] as an input and outputs the final state after performing
        quantum operation
        - n (int): the number of wires of circuit

    Returns:
        - (bool): True / False according to whether the input circuit preserves the number of particles or not
    """

    # QHACK #
    states = basis_states(n+1)
    for state in states:
        basis = circuit(state).numpy()
        print(basis)
        count1 = 0
        count2 = 0
        for i in range(n):
            if state[i] == 1:
                count1 += 1
            if basis[i] == 1:
                count2 += 1
        if count1 != count2:
            return False
            break
    return True

In [ ]:
@qml.qnode(dev)
def circ(gate_list, wire_list, param_list, state):
    qml.BasisState(np.array(state), wires=range(n))
    j = 0
    for i in range(len(gate_list)):
        gate = getattr(qml, str(gate_list[i]))
        if "non_parametric_ops" not in gate.__module__.split("."):
            gate(*param_list[j], wires=[int(w) for w in wire_list[i]])
            j += 1
        else:
            gate(wires=[int(w) for w in wire_list[i]])
    return qml.state()

def circuit(state):
    return circ(gate_list, wire_list, param_list, state)

In [ ]:
gate_list = ['DoubleExcitation', 'SingleExcitation'] 
wire_list = [[0, 1, 2, 3], [0, 1]] 
param_list = [[0.732], [1.]]
n = 4

basis_states(n)


[array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0

In [ ]:
n = 4
dev = qml.device('default.qubit', wires=n+1)
@qml.qnode(dev)
def circuit(list1, n):
    qml.BasisState(np.array(list1), wires=range(n+1))
    return qml.state()

array = []
for m in range(2**n+1):
  y = circuit(binary_list(m, n+1), n+1)
  print(y)
  array.append(y.numpy())

WireError: ignored

In [ ]:
circ(gate_list, wire_list, param_list, [0, 0, 1])

ValueError: ignored

In [ ]:
basis_states(5)

[array([1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j]),
 array([0.+0.j, 0.+0.j, 0.+0.j, 1.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j,
        0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.+0.j, 0.

# Quantum Chemistry 200

In [ ]:
def check_simplification(op1, op2):
    """As we have seen in the problem statement, given two Pauli operators, you could obtain the expected value
    of each of them by running a single circuit following the two defined rules. This function will determine whether,
    given two Pauli operators, such a simplification can be produced.

    Args:
        - op1 (list(str)): First Pauli word (list of Pauli operators), e.g., ["Y", "I", "Z", "I"].
        - op2 (list(str)): Second Pauli word (list of Pauli operators), e.g., ["Y", "I", "X", "I"].

    Returns:
        - (bool): 'True' if we can simplify them, 'False' otherwise. For the example args above, the third qubit does not allow simplification, so the function would return `False`.
    """

    # QHACK
    for i in range(len(op1)):
        if op1[i] == op2[i]:
          continue
        if op1[i] != op2[i]:
            if op1[i] == 'I' or op2[i] == 'I':
                continue
            else:
                return False
    return True

In [ ]:
check_simplification(['Z','I','I','I'],['Z','Y','I','I'])
obs = [['Z','X','I','I'],['Z','Y','I','I'],['X','I','X','Y'], ['I','Z','I','I']]
check_simplification(obs[0], obs[1])

False

In [ ]:
def join_operators(op1, op2):
    """This function will receive two operators that we know can be simplified
    and returns the operator corresponding to the union of the two previous ones.

    Args:
        - op1 (list(str)): First Pauli word (list of Pauli operators), e.g., ["Y", "I", "Z", "I"].
        - op2 (list(str)): Second Pauli word (list of Pauli operators), e.g., ["Y", "X", "I", "I"].

    Returns:
        - (list(str)): Pauli operator corresponding to the union of op1 and op2.
        For the case above the output would be ["Y", "X", "Z", "I"]
    """

    # QHACK
    operators = []
    for i in range(len(op1)):
      if op1[i] == op2[i]:
          operators.append(op1[i])
      if op1[i] != op2[i]:
        if op1[i] == 'I':
          operators.append(op2[i])
        else:
          operators.append(op1[i])
    return operators
    # QHACK

In [ ]:
def optimize_measurements(obs_hamiltonian):
    """This function will go through the list of Pauli words provided in the statement, grouping the operators
    following the simplification process of the previous functions.

    Args:
        - obs_hamiltonian (list(list(str))): Groups of Pauli words making up the Hamiltonian.

    Returns:
        - (list(list(str))): The chosen Pauli operators to measure after grouping.
    """

    final_solution = []

    for op1 in obs_hamiltonian:
        added = False
        for i, op2 in enumerate(final_solution):

            if check_simplification(op1, op2):
                final_solution[i] = join_operators(op1, op2)
                added = True
                break
        if not added:
            final_solution.append(op1)

    return final_solution

In [ ]:
obs = [['Z','X','I','I'],['Z','Y','I','I'],['X','I','X','Y'], ['I','Z','I','I']]
obs1 = [['Z','I','I','I'],['Z','Y','I','I'],['X','I','X','Y'],['I','Z','I','I']]
optimize_measurements(obs)

[['Z', 'X', 'I', 'I'], ['Z', 'Y', 'I', 'I'], ['X', 'Z', 'X', 'Y']]

In [ ]:
def compression_ratio(obs_hamiltonian, final_solution):
    """Function that calculates the compression ratio of the procedure.

    Args:
        - obs_hamiltonian (list(list(str))): Groups of Pauli operators making up the Hamiltonian.
        - final_solution (list(list(str))): Your final selection of observables.

    Returns:
        - (float): Compression ratio your solution.
    """

    # QHACK
    r = 1 - len(final_solution)/len(obs_hamiltonian)
    return r
    # QHACK

In [ ]:
compression_ratio(obs, [['X', 'Z', 'X', 'Y']])

0.75

#Quantum Chemistry 300

In [ ]:
def givens_rotations(a, b, c, d):
    """Calculates the angles needed for a Givens rotation to out put the state with amplitudes a,b,c and d

    Args:
        - a,b,c,d (float): real numbers which represent the amplitude of the relevant basis states (see problem statement). Assume they are normalized.

    Returns:
        - (list(float)): a list of real numbers ranging in the intervals provided in the challenge statement, which represent the angles in the Givens rotations,
        in order, that must be applied.
    """

    x = 2 * np.arcsin(-b)
    y = - 2 * np.arcsin(c/np.cos(x/2))
    z = 2 * np.arccos(a / (np.cos(x/2)*np.cos(y/2)))
    
    return x, y, z

# Quantum Chemistry 500

In [ ]:
dev = qml.device("default.qubit", wires=4)
symbols = ["H", "H"]
geometry = np.array([[0.0, 0.0, -0.6], [0.0, 0.0, 0.6]], requires_grad=False)
mol = hf.Molecule(symbols, geometry)
H = hf.generate_hamiltonian(mol)()
electrons = 2
theta = np.array(0.0, requires_grad=True)
opt = qml.GradientDescentOptimizer(stepsize=0.4)

In [ ]:
wires = 4
def circuit(param, wires):
    qml.BasisState(np.array([1, 1, 0, 0]), wires=wires)
    qml.DoubleExcitation(param, wires=[0, 1, 2, 3])

dev = qml.device("default.qubit", wires=wires)
@qml.qnode(dev)
def cost_fn(param):
    circuit(param, wires=range(wires))
    return qml.expval(H)


def ground_state_VQE(H):
    """Perform VQE to find the ground state of the H2 Hamiltonian.

    Args:
        - H (qml.Hamiltonian): The Hydrogen (H2) Hamiltonian

    Returns:
        - (float): The ground state energy
        - (np.ndarray): The ground state calculated through your optimization routine
    """
    #define the initial theta
    theta = np.array(0.0, requires_grad=True)

    # store the values of the cost function
    # expectation value of H in terms of theta
    energy = [cost_fn(theta)]

    # store the values of the circuit parameter
    angle = [theta]

    max_iterations = 100
    conv_tol = 1e-06

    # optimizer
    opt = qml.GradientDescentOptimizer(stepsize=0.4)

    for n in range(max_iterations):
        #get the new value of energy and theta
        theta, prev_energy = opt.step_and_cost(cost_fn, theta)

        # append the values
        energy.append(cost_fn(theta))
        angle.append(theta)

        #check accuracy
        conv = np.abs(energy[-1] - prev_energy)

        if conv <= conv_tol:
            return energy[-1], angle[-1]

In [ ]:
ground_state_VQE(H)

(tensor(-1.12669837, requires_grad=True),
 tensor(0.18494701, requires_grad=True))

#Pennylane 101 100

In [ ]:
dev = qml.device('default.qubit', wires=2)
@qml.qnode(dev)
def circuit(angles):
  qml.RX(angles[0], wires=0)
  qml.RY(angles[1], wires=0)

  qml.RY(angles[1], wires=1)
  qml.RX(angles[0], wires=1)

  sigma1 = qml.expval(qml.PauliX(wires = 0))
  sigma2 = qml.expval(qml.PauliX(wires = 1))
  
  return sigma1

def compare_circuits(angles):
  """Given two angles, compare two circuit outputs that have their order of operations flipped: RX then RY VERSUS RY then RX.

  Args:
      - angles (np.ndarray): Two angles

  Returns:
      - (float): | < \sigma^x >_1 - < \sigma^x >_2 |
  """

  sigma = circuit(angles).numpy()

  return abs(sigma[0]- sigma[1])

#Pennylane 101 200

In [ ]:
def matrix_norm(mixed_state, pure_state):
    """Computes the matrix one-norm of the difference between mixed and pure states.

    Args:
        - mixed_state (np.tensor): A density matrix
        - pure_state (np.tensor): A pure state

    Returns:
        - (float): The matrix one-norm
    """

    return np.sum(np.abs(mixed_state - np.outer(pure_state, np.conj(pure_state))))

def compare_circuits(num_wires, params):
    """Function that returns the matrix norm between the mixed- and pure-state versions of the same state.

    Args:
        - num_wires (int): The number of qubits / wires
        - params (list(np.ndarray)): Two arrays with num_wires floats that correspond to angles of y-rotations
        for each wire

    Returns:
        - mat_norm (float): The matrix one-norm
    """

    # QHACK #

    dev = qml.device('default.qubit', wires=num_wires)
    @qml.qnode(dev)
    def pure_circuit():
        """A circuit that contains `num_wires` y-rotation gates.
        The argument params[0] are the parameters you should use here to define the y-rotations.

        Returns:
            - (np.tensor): A state vector
        """
        # create the circuit here
        for i in range(num_wires):
          qml.RY(params[0][i], wires = i)
        return qml.state()

    dev = qml.device('default.mixed', wires=num_wires)
    @qml.qnode(dev)
    def mixed_circuit():
        """A circuit that contains `num_wires` y-rotation gates.
        The argument params[1] are the parameters you should use here to define the y-rotations.

        Returns:
            - (np.tensor): A density matrix
        """
        # create the circuit here
        for i in range(num_wires):
          qml.RY(params[1][i], wires = i)
        return qml.state()
  

    # QHACK #

    # DO NOT MODIFY any of the next lines in this scope
    mixed_state = mixed_circuit()
    pure_state = pure_circuit()
    mat_norm = matrix_norm(mixed_state, pure_state)

    return mat_norm

#Pennylane 101 300

In [ ]:
dev = qml.device("default.qubit", wires=2)
@qml.qnode(dev)
def superdense_coding(bits, alpha):
    """Construct a quantum circuit that implements superdense coding, given a not necessarily maximally entangled state

    Args:
        - bits (int): 0 (binary: 00), 1 (binary: 01), 2 (binary: 10), or 3 (binary: 11), Alice's bits that she wants to communicate to Bob.
        - alpha (float): angle parametrizing the entangled state

    Returns:
        - (np.tensor): Probability that Bob will guess Alice's bits correctly
    """

    # QHACK #

    # Prepare entangled state here
    qml.RY(alpha*2, wires = 0)
    qml.CNOT(wires=[0, 1])

    # Implement Alice's operations on her qubit here
    if bits == 1:
        qml.PauliX(wires = 0)
    if bits == 2:
        qml.PauliZ(wires = 0)
    if bits == 3:
        qml.PauliX(wires = 0)
        qml.PauliZ(wires = 0)

    # Implement Bob's measurement procedure here
    qml.CNOT(wires=[0, 1])
    qml.Hadamard(wires = 0)


    # QHACK #

    return qml.probs(wires=[0, 1])

In [ ]:
def return_probs(bits, alpha):
    """Returns the output of the superdense_coding function for a given index (bits)"""
    # DO NOT MODIFY anything in this code block
    return superdense_coding(bits, alpha)[bits].numpy()

#Pennylane 101 500

In [ ]:
def error_wire(circuit_output):
    """Function that returns an error readout.

    Args:
        - circuit_output (?): the output of the `circuit` function.

    Returns:
        - (np.ndarray): a length-4 array that reveals the statistics of the
        error channel. It should display your algorithm's statistical prediction for
        whether an error occurred on wire `k` (k in {1,2,3}). The zeroth element represents
        the probability that a bitflip error does not occur.

        e.g., [0.28, 0.0, 0.72, 0.0] means a 28% chance no bitflip error occurs, but if one
        does occur it occurs on qubit #2 with a 72% chance.
    """

    # QHACK #

    return circuit_output.numpy()

    # QHACK #


dev = qml.device("default.mixed", wires=3)


@qml.qnode(dev)
def circuit(p, alpha, tampered_wire):
    """A quantum circuit that will be able to identify bitflip errors.

    DO NOT MODIFY any already-written lines in this function.

    Args:
        p (float): The bit flip probability
        alpha (float): The parameter used to calculate `density_matrix(alpha)`
        tampered_wire (int): The wire that may or may not be flipped (zero-index)

    Returns:
        Some expectation value, state, probs, ... you decide!
    """

    qml.QubitDensityMatrix(density_matrix(alpha), wires=[0, 1, 2])

    # QHACK #

    # put any input processing gates here
    qml.CNOT(wires = [0, 1])
    qml.CNOT(wires = [0, 2])

    qml.BitFlip(p, wires=int(tampered_wire))

    # put any gates here after the bitflip error has occurred
    qml.CNOT(wires = [0, 1])
    qml.CNOT(wires = [0, 2])
    qml.Toffoli(wires = [2,1,0])

    return qml.probs(wires=[0,1,2])
    
    # QHACK #

def density_matrix(alpha):
    """Creates a density matrix from a pure state."""
    # DO NOT MODIFY anything in this code block
    psi = alpha * np.array([1, 0], dtype=float) + np.sqrt(1 - alpha**2) * np.array(
        [0, 1], dtype=float
    )
    psi = np.kron(psi, np.array([1, 0, 0, 0], dtype=float))
    return np.outer(psi, np.conj(psi))

In [ ]:
error_wire(circuit(0.8, 0.1234, 0))

array([0.00304551, 0.        , 0.        , 0.01218205, 0.19695449,
       0.        , 0.        , 0.78781795])

#Algorithm 100


In [ ]:
import sys
import pennylane as qml
from pennylane import numpy as np


def deutsch_jozsa(oracle):
    """This function will determine whether an oracle defined by a function f is constant or balanced.

    Args:
        - oracle (function): Encoding of the f function as a quantum gate. The first two qubits refer to the input and the third to the output.

    Returns:
        - (str): "constant" or "balanced"
    """

    dev = qml.device("default.qubit", wires=3, shots=1)

    @qml.qnode(dev)
    def circuit():
        """Implements the Deutsch Jozsa algorithm."""

        # QHACK #

        # Insert any pre-oracle processing here
        
        qml.Hadamard(wires=0)
        qml.Hadamard(wires=1)
        qml.X(wires=2)
        qml.Hadamard(wires=2)
        qml.CNOT(wires=[0,2])
        #oracle()  # DO NOT MODIFY this line

        # Insert any post-oracle processing here
        qml.Hadamard(wires=0)
        qml.Hadamard(wires=1)
        # QHACK #

        return qml.sample(wires=range(2))

    sample = circuit()
    print(sample)

    # QHACK #

    # From `sample` (a single call to the circuit), determine whether the function is constant or balanced.
    if(sample[0] == sample[1]):
      return 'constant'
    else:
      return 'balanced'
    # QHACK #

In [ ]:
numbers = [1]
def oracle():
  for i in numbers:
    qml.CNOT(wires=[i,2])
output = deutsch_jozsa(oracle)
print(output)

[0 0]
constant


#algorithms 200

In [ ]:
graph = {
    0: [1],
    1: [0, 2, 3, 4],
    2: [1],
    3: [1],
    4: [1, 5, 7, 8],
    5: [4, 6],
    6: [5, 7],
    7: [4, 6],
    8: [4],
}


def n_swaps(cnot):
    """Count the minimum number of swaps needed to create the equivalent CNOT.

    Args:
        - cnot (qml.Operation): A CNOT gate that needs to be implemented on the hardware
        You can find out the wires on which an operator works by asking for the 'wires' attribute: 'cnot.wires'

    Returns:
        - (int): minimum number of swaps
    """

    # QHACK #

    if(cnot.wires[0]<cnot.wires[1]):
      source = cnot.wires[1]
      destination = cnot.wires[0]
    elif(cnot.wires[0]>cnot.wires[1]):
      source = cnot.wires[0]
      destination = cnot.wires[1]
    
    def depth_first_search(graph, source , destination):
      size = len(graph)
      colour = ["black"] * size
      
      where_come_from = [-1] * size
      
      def go(current_node):
          colour[current_node] = "blue "
          for i in graph[current_node]:
              if colour[i] == "black":
                  where_come_from[i] = current_node
                  go(i)
      go(source)
      
      if where_come_from[destination] == -1:
          return[]
      
      path = [destination]
      
      while path[-1] != source:
          u = path[-1]
          v = where_come_from[u]
          path.append(v)
      
      path = list(reversed(path))
      return path
    
    path = depth_first_search(graph,source,destination)
    return (len(path)-2)*2

    # QHACK #

In [ ]:
any(x in graph[0] for x in graph[4])
graph.keys()

dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8])

In [ ]:
inputs = 1, 0
output = n_swaps(qml.CNOT(wires=[int(i) for i in inputs]))
print(output)

0


In [ ]:
def depth_first_search(graph, source , destination):
    size = len(graph)
    colour = ["black"] * size
    
    where_come_from = [-1] * size
    
    def go(current_node):
        colour[current_node] = "blue "
        for i in graph[current_node]:
            if colour[i] == "black":
                where_come_from[i] = current_node
                go(i)
    go(source)
    
    if where_come_from[destination] == -1:
        return[]
    
    path = [destination]
    
    while path[-1] != source:
        u = path[-1]
        v = where_come_from[u]
        path.append(v)
    
    path = list(reversed(path))
    return path

In [ ]:
depth_first_search(graph,6,8)

[6, 5, 4, 8]